In [10]:
!pip install textblob

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
!pip install wordcloud

In [13]:
!pip install cufflinks

In [33]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
cf.go_offline();
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

pd.set_option('display.max_columns',None)


In [28]:
df=pd.read_csv("amazon.csv")

In [16]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [17]:
df.tail()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4910,4910,"ZM ""J""",1,I bought this Sandisk 16GB Class 10 to use wit...,23-07-2013,503,0,0,0,0,0.0,0.0
4911,4911,Zo,5,Used this for extending the capabilities of my...,22-08-2013,473,0,0,0,0,0.0,0.0
4912,4912,Z S Liske,5,Great card that is very fast and reliable. It ...,31-03-2014,252,0,0,0,0,0.0,0.0
4913,4913,Z Taylor,5,Good amount of space for the stuff I want to d...,16-09-2013,448,0,0,0,0,0.0,0.0
4914,4914,Zza,5,I've heard bad things about this 64gb Micro SD...,01-02-2014,310,0,0,0,0,0.0,0.0


In [29]:
df=df.sort_values("wilson_lower_bound",ascending=False)
df.drop('Unnamed: 0',inplace=True,axis=1)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [19]:
def missing_values_analysis(df):
  na_columns = [col for col in df.columns if df[col].isnull().sum() >0]
  n_miss = df[na_columns].isnull().sum().sort_values(ascending=True)
  ratio=(df[na_columns].isnull().sum() / df.shape[0]* 100).sort_values(ascending=True)
  missing_df = pd.concat([n_miss,np.round(ratio,2)], axis=1, keys=['Missing Values','Ratio'])
  missing_df=pd.DataFrame(missing_df)
  return missing_df
def check_dataframe(df,head=5,tail=5):
  print("SHAPE".center(82,'~'))
  print('Rows:{}'.format(df.shape[0]))
  print('column:{}'.format(df.shape[1]))
  print("TYPES".center(82,'~'))
  print(df.dtypes)
  print("".center(82,'~'))
  print(missing_values_analysis(df))
  print('DUPLICATED VALUES'.center(82,'~'))
  print(df.quantile([0,0.05,0.50,0.95,0.99,1], numeric_only=True).T)

check_dataframe(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows:4915
column:11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00
overall                 1.0   2.0    5.0    5.000000    5.00000     5.000000
day_diff            

In [20]:
def check_class(dataframe):
  nunique_df=pd.DataFrame({'Variable':dataframe.columns,
                           'Classes':[dataframe[i].nunique() \
                                      for i in dataframe.columns]})
  nunique_df=nunique_df.sort_values('Classes', ascending=False)
  unique_df=nunique_df.reset_index(drop=True)
  return nunique_df

check_class(df)

,Variable,Classes
2,reviewText,4912
0,reviewerName,4594
3,reviewTime,690
4,day_diff,690
10,wilson_lower_bound,40
9,score_average_rating,28
8,score_pos_neg_diff,27
7,total_vote,26
5,helpful_yes,23
6,helpful_no,17


In [34]:
constraints = ['#B34D22','#EBE00C','#1FEB0C','#0C92EB','#EB0CD5']
def categorical_variable_summary(df, column_name):
  fig = make_subplots(rows=1,cols=2,
                      subplot_titles=('Countplot','Percentage'),
                      specs=[[{"type":"xy"},{'type':'domain'}]])
  fig.add_trace(go.Bar(y=df[column_name].value_counts().values.tolist(),
                       x=[str(i) for i in df[column_name].value_counts().index],
                       text = df[column_name].value_counts().values.tolist(),
                       textfont =dict(size=14),
                       name = column_name,
                       textposition = 'auto',
                       showlegend=False,
                       marker=dict(color=constraints,
                                   line=dict(color='#DBE6EC',
                                             width=1))),
                row=1,col=1)
  fig.add_trace(go.Pie(labels=df[column_name].value_counts().keys(),
                       values=df[column_name].value_counts().values,
                       textfont = dict(size = 18),
                       textposition='auto',
                       showlegend=False,
                       name=column_name,
                       marker = dict(colors = constraints)),
                row=1,col=2)
  fig.update_layout(title={'text':column_name,
                           'y':0.9,
                           'x':0.5,
                           'xanchor':'center',
                           'yanchor':'top'},
                    template='plotly_white')
  return fig

fig_overall = categorical_variable_summary(df, 'overall')

In [22]:
df.reviewText.head()

,reviewText
2031,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449,I have tested dozens of SDHC and micro-SDHC ca...
4212,NOTE: please read the last update (scroll to ...
317,"If your card gets hot enough to be painful, it..."
4672,Sandisk announcement of the first 128GB micro ...


In [23]:
review_example=df.reviewText[2031]
review_example

'[[ UPDATE - 6/19/2014 ]]So my lovely wife bought me a Samsung Galaxy Tab 4 for Father\'s Day and I\'ve been loving it ever since.  Just as other with Samsung products, the Galaxy Tab 4 has the ability to add a microSD card to expand the memory on the device.  Since it\'s been over a year, I decided to do some more research to see if SanDisk offered anything new.  As of 6/19/2014, their product lineup for microSD cards from worst to best (performance-wise) are the as follows:SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow, the difference between all of these cards are simply the speed in which you can read/write data to the card.  Yes, the published rating of most all these cards (except the SanDisk regular) are Class 10/UHS-I but that\'s just a rating... Actual real world performance does get better with each model, but with faster cards come more expensive prices.  Since Amazon doesn\'t carry the Ultra PLUS model of microSD card, I had 

In [24]:
review_example=re.sub("[^a-zA-Z]",'',review_example)
review_example

'UPDATESomylovelywifeboughtmeaSamsungGalaxyTabforFathersDayandIvebeenlovingiteversinceJustasotherwithSamsungproductstheGalaxyTabhastheabilitytoaddamicroSDcardtoexpandthememoryonthedeviceSinceitsbeenoverayearIdecidedtodosomemoreresearchtoseeifSanDiskofferedanythingnewAsoftheirproductlineupformicroSDcardsfromworsttobestperformancewisearetheasfollowsSanDiskSanDiskUltraSanDiskUltraPLUSSanDiskExtremeSanDiskExtremePLUSSanDiskExtremePRONowthedifferencebetweenallofthesecardsaresimplythespeedinwhichyoucanreadwritedatatothecardYesthepublishedratingofmostallthesecardsexcepttheSanDiskregularareClassUHSIbutthatsjustaratingActualrealworldperformancedoesgetbetterwitheachmodelbutwithfastercardscomemoreexpensivepricesSinceAmazondoesntcarrytheUltraPLUSmodelofmicroSDcardIhadtododirectcomparisonsbetweentheSanDiskUltraExtremeandExtremePLUSAsmentionedinmyearlierreviewIpurchasedtheSanDiskUltraformyGalaxySMyquestionwasdidIwanttopayovermoreforacardthatisfasterthantheoneIalreadyownedOrIcouldpayalmostdoubletoget

In [16]:
review_example = df.reviewText[2031]
review_example = re.sub("[^a-zA-Z]",'',review_example)
review_example = review_example.lower().split()

In [25]:
review_example

'UPDATESomylovelywifeboughtmeaSamsungGalaxyTabforFathersDayandIvebeenlovingiteversinceJustasotherwithSamsungproductstheGalaxyTabhastheabilitytoaddamicroSDcardtoexpandthememoryonthedeviceSinceitsbeenoverayearIdecidedtodosomemoreresearchtoseeifSanDiskofferedanythingnewAsoftheirproductlineupformicroSDcardsfromworsttobestperformancewisearetheasfollowsSanDiskSanDiskUltraSanDiskUltraPLUSSanDiskExtremeSanDiskExtremePLUSSanDiskExtremePRONowthedifferencebetweenallofthesecardsaresimplythespeedinwhichyoucanreadwritedatatothecardYesthepublishedratingofmostallthesecardsexcepttheSanDiskregularareClassUHSIbutthatsjustaratingActualrealworldperformancedoesgetbetterwitheachmodelbutwithfastercardscomemoreexpensivepricesSinceAmazondoesntcarrytheUltraPLUSmodelofmicroSDcardIhadtododirectcomparisonsbetweentheSanDiskUltraExtremeandExtremePLUSAsmentionedinmyearlierreviewIpurchasedtheSanDiskUltraformyGalaxySMyquestionwasdidIwanttopayovermoreforacardthatisfasterthantheoneIalreadyownedOrIcouldpayalmostdoubletoget

In [30]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', '', text) # Remove HTML tags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\d+', '', text) # Remove numbers
    text = re.sub(r'[^a-zA-Z\s.,!?”\"]', '', text) # Keep letters, spaces, and common punctuation
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

df["reviewText"] = df["reviewText"].apply(clean_text)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife bought me a samsung g...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and microsdhc car...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to the...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"if your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro sd ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [31]:
nltk.download('vader_lexicon') # Ensure vader lexicon is downloaded

df[['polarity', 'subjectivity']]=df['reviewText'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in df.iterrows():
    score=SentimentIntensityAnalyzer().polarity_scores(row['reviewText'])

    neg=score['neg']
    neu=score['neu']
    pos=score['pos']

    if neg > pos:
        df.loc[index,'sentiment'] = "Negative"
    elif pos > neg:
        df.loc[index,'sentiment']="Positive"
    else:
        df.loc[index,'sentiment']="Neutral"

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [43]:
sentiment_overall_avg = df.groupby('sentiment')['overall'].mean().reset_index()
display(sentiment_overall_avg)

,sentiment,overall
0,Negative,3.950907
1,Neutral,4.649485
2,Positive,4.744508


In [55]:
sentiment_overall_avg = df.groupby('sentiment')['overall'].mean().reset_index()
fig = go.Figure(data=[go.Bar(x=sentiment_overall_avg['sentiment'], y=sentiment_overall_avg['overall'],
                            marker_color=constraints,
                            text=round(sentiment_overall_avg['overall'],2),
                            textposition='auto')])

fig.update_layout(title_text='Average Overall Rating by Sentiment',
                  xaxis_title='Sentiment',
                  yaxis_title='Average Overall Rating',
                  template='plotly_white')
iplot(fig)

In [45]:
!pip install plotly==5.20.0

In [46]:
import plotly.io as pio
pio.renderers.default = 'colab'


In [47]:
fig_sentiment = categorical_variable_summary(df, 'sentiment')

In [36]:
avg_polarity = df['polarity'].mean()
avg_subjectivity = df['subjectivity'].mean()

print(f"Average Polarity: {avg_polarity:.4f}")
print(f"Average Subjectivity: {avg_subjectivity:.4f}")

Average Polarity: 0.2773
Average Subjectivity: 0.5395


In [48]:
df[df['sentiment']=='Positive'].sort_values("wilson_lower_bound",
                                            ascending=False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife bought me a samsung g...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.156702,0.558574,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and microsdhc car...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.091564,0.514756,Positive
4212,SkincareCEO,1,note please read the last update scroll to the...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.208749,0.505531,Positive
317,"Amazon Customer ""Kelly""",1,"if your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.151042,0.470566,Positive
4672,Twister,5,sandisk announcement of the first gb micro sd ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.164206,0.493237,Positive


In [50]:
fig_sentiment_2 = categorical_variable_summary(df,'sentiment')

# Task
Display the 'overall' ratings distribution, followed by the 'sentiment' distribution (twice) using `iplot()`.

## Display Overall Rating Distribution

### Subtask:
Modify the code to display the 'overall' ratings distribution using `iplot(fig_overall)`.


**Reasoning**:
The subtask explicitly asks to display the 'overall' ratings distribution using `iplot(fig_overall)`. This command will render the Plotly figure that was previously generated.



In [51]:
iplot(fig_overall)

## Display Sentiment Distribution

### Subtask:
Modify the code to display the 'sentiment' distribution using `iplot(fig_sentiment)`.


**Reasoning**:
The subtask explicitly asks to display the 'sentiment' distribution using `iplot(fig_sentiment)`, which was already generated in a previous step.



In [52]:
iplot(fig_sentiment)

**Reasoning**:
The subtask is to display the 'sentiment' distribution using `iplot(fig_sentiment)` again, which was already prepared in a previous step. This is the final display requirement of the subtask and main task.



In [53]:
iplot(fig_sentiment)

## Display Second Sentiment Distribution

### Subtask:
Display the second 'sentiment' distribution using `iplot(fig_sentiment_2)`.


**Reasoning**:
The subtask explicitly asks to display the 'sentiment' distribution using `iplot(fig_sentiment_2)`. This command will render the Plotly figure that was previously generated.



In [54]:
iplot(fig_sentiment_2)

## Final Task

### Subtask:
Confirm that the pie charts are now visible and summarize the relevant plots.


## Summary:

### Data Analysis Key Findings
*   The 'overall' ratings distribution was successfully displayed using `iplot(fig_overall)`.
*   The initial 'sentiment' distribution was successfully displayed twice using `iplot(fig_sentiment)`, as requested.
*   A second 'sentiment' distribution, `fig_sentiment_2`, was also successfully displayed using `iplot(fig_sentiment_2)`.
*   All distributions were rendered interactively using Plotly's `iplot()` function.

### Insights or Next Steps
*   Review the generated pie charts to visually confirm the distribution patterns for overall ratings and sentiment.
*   Utilize these visualizations to identify trends or anomalies in customer feedback, which can inform product development or service improvements.
